In [1]:
from scvi.dataset import EbiData, MouseAtlas, UnionDataset, AnnDatasetFromAnnData
from Eval_basis import *
import scanpy as sc
import pandas as pd
import scipy.sparse as sparse

[2019-08-23 18:32:53,073] INFO - scvi._settings | Added StreamHandler with custom formatter to 'scvi' logger.
/home/michael/anaconda3/envs/scvi/lib/python3.7/site-packages/sklearn/utils/linear_assignment_.py:21: DeprecationWarning: The linear_assignment_ module is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  DeprecationWarning)


In [6]:
ebi_1 = EbiData("./data", 'E-MTAB-6946', result_file='raw')

/home/michael/anaconda3/envs/scvi/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (8,20) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
[2019-08-23 18:41:49,519] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2019-08-23 18:41:49,524] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2019-08-23 18:41:52,554] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2019-08-23 18:41:55,489] INFO - scvi.dataset.dataset | Downsampled from 167276 to 167276 cells


In [7]:
ebi_2 = EbiData("./data", 'E-MTAB-7320', result_file='raw')

[2019-08-23 18:42:25,408] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2019-08-23 18:42:25,410] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2019-08-23 18:42:26,535] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2019-08-23 18:42:27,597] INFO - scvi.dataset.dataset | Downsampled from 62848 to 62848 cells


In [8]:
fpaths_and_fnames = {'data': "./data/mouse_atlas/cleaned_data_sparse.npz",
                     'cell': "./data/mouse_atlas/cell_annotation.csv",
                     'gene': "./data/mouse_atlas/gene_annotation.csv",
                     'pheno': "./data/mouse_atlas/phenotype_data.csv"}
mouse_atlas = MouseAtlas("./data/mouse_atlas",
                   fpaths_and_fnames,
                   True,
                   False)

In [2]:
conv = pd.read_csv("./data/gene_maps/hugo_mouse_genes-proteincoding.csv", header=0, index_col=0)
conv.index = conv.index.str.lower()

data_path = os.path.join(("./data"))
mouse_data_path = os.path.join(data_path, "mouse_data")
dsets = [] 
for file in os.listdir(f"{data_path}/mouse_data"):
#     if "droplet" in file:
    dset = sc.read_h5ad(os.path.join(mouse_data_path, file))
    dset.obs.rename(columns={"cell_ontology_class": "cell_types"}, inplace=True)
    
    dset = AnnDatasetFromAnnData(dset)
    
    gns_conved = conv.reindex(np.char.lower(dset.gene_names))["ensembl"]
    if not isinstance(dset.X, np.ndarray):
        X = dset.X.toarray()
    else:
        X = dset.X
    mask = ~gns_conved.isnull()
    
    dset.gene_names = gns_conved[mask].values.astype(str)
    dset.X = X[:, mask]
    dset.cell_types = np.array([ct.replace("ï", "i") for ct in dset.cell_types])
    
    dsets.append(dset)

[2019-08-23 18:32:57,362] INFO - scvi.dataset.anndataset | Dense size under 1Gb, casting to dense format (np.ndarray).
[2019-08-23 18:32:57,891] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2019-08-23 18:32:57,893] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2019-08-23 18:32:58,064] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2019-08-23 18:32:58,246] INFO - scvi.dataset.dataset | Downsampled from 7916 to 7916 cells
[2019-08-23 18:32:59,208] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2019-08-23 18:33:00,390] INFO - scvi.dataset.anndataset | Dense size under 1Gb, casting to dense format (np.ndarray).
[2019-08-23 18:33:00,681] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2019-08-23 18:33:00,682] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2019-08-23 18:33:00,760] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2019-08-23 18:33:00,842] IN

[2019-08-23 18:34:05,347] INFO - scvi.dataset.anndataset | Dense size under 1Gb, casting to dense format (np.ndarray).
[2019-08-23 18:34:05,570] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2019-08-23 18:34:05,571] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2019-08-23 18:34:05,612] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2019-08-23 18:34:05,653] INFO - scvi.dataset.dataset | Downsampled from 1561 to 1561 cells
[2019-08-23 18:34:05,814] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2019-08-23 18:34:06,548] INFO - scvi.dataset.anndataset | Dense size under 1Gb, casting to dense format (np.ndarray).
[2019-08-23 18:34:06,847] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2019-08-23 18:34:06,848] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2019-08-23 18:34:06,919] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2019-08-23 18:34:06,994] IN

[2019-08-23 18:34:41,831] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2019-08-23 18:34:42,065] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2019-08-23 18:34:42,306] INFO - scvi.dataset.dataset | Downsampled from 9734 to 9734 cells
[2019-08-23 18:34:43,642] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2019-08-23 18:34:47,283] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2019-08-23 18:34:47,285] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2019-08-23 18:34:47,657] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2019-08-23 18:34:48,075] INFO - scvi.dataset.dataset | Downsampled from 29322 to 29322 cells
[2019-08-23 18:34:52,237] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2019-08-23 18:34:56,632] INFO - scvi.dataset.anndataset | Dense size under 1Gb, casting to dense format (np.ndarray).
[2019-08-23 18:34:57,078] INFO - sc

In [3]:
mouse_muris_senis = UnionDataset("./data", 
                                 gene_map_load_filename="gene_maps/ensembl_mouse_genes-proteincoding", 
                                 low_memory=False)

In [4]:
mouse_muris_senis.join_datasets(data_source="memory", 
                                data_target="memory",
                                gene_datasets=dsets)
mouse_muris_senis.name = "Tabula Muris Senis"

Concatenating datasets: 100%|██████████| 38/38 [04:53<00:00,  6.43s/it]
[2019-08-23 18:40:05,167] INFO - scvi.dataset.dataset | Union dataset is set to ignore batch annotation.
100%|██████████| 290905/290905 [00:28<00:00, 10324.15it/s]
[2019-08-23 18:40:39,087] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2019-08-23 18:40:39,115] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2019-08-23 18:40:39,122] INFO - scvi.dataset.dataset | Union dataset is set to ignore batch annotation.
[2019-08-23 18:40:39,126] INFO - scvi.dataset.dataset | Joined 38 datasets to one of shape 290905 x 22250.


In [5]:
dsets = None
mouse_muris_senis.cell_types = np.array([ct.replace("ï", "i") for ct in mouse_muris_senis.cell_types])

In [9]:
complete_mouse = UnionDataset("./data", 
                              gene_map_load_filename="gene_maps/ensembl_mouse_genes-proteincoding", 
                              low_memory=False)

In [ ]:
complete_mouse.join_datasets(data_source="memory", 
                             data_target="loom", 
                             out_filename="mouse_data_all.loom", 
                             gene_datasets=[mouse_muris_senis, ebi_1, ebi_2, mouse_atlas])

Writing dataset 'UnionDataset - Tabula Muris Senis' to file:   0%|          | 0/4 [00:00<?, ?it/s]
Writing sparse matrix iteratively to file...:  62%|██████▏   | 21/34 [01:28<00:54,  4.19s/it]


Writing sparse matrix iteratively to file...:  21%|██        | 55/267 [03:53<15:14,  4.31s/it]


Writing sparse matrix iteratively to file...:  52%|█████▏    | 138/267 [09:49<09:10,  4.26s/it]


Writing sparse matrix iteratively to file...:  82%|████████▏ | 220/267 [15:45<03:21,  4.30s/it]


Writing sparse matrix iteratively to file...:  83%|████████▎ | 221/267 [15:50<03:17,  4.29s/it]

In [ ]:
complete_mouse = UnionDataset("./data", 
                              gene_map_load_filename="gene_maps/ensembl_mouse_genes-proteincoding", 
                              data_load_filename="mouse_data_all.loom",
                              low_memory=True)

In [ ]:
complete_mouse.gene_names

In [ ]:
n_epochs = 100
colors=None

print("Training VAE")

trainer = train_vae(complete_mouse, "./data", f"max_data_model", n_epochs=n_epochs)
# trainer_small = train_vae(data_small, "./data", f"small_{tissue}_data_portion", n_epochs=n_epochs)



In [ ]:
ebi_with_celltypes = EbiData("./data")

In [ ]:
dot_size = (mpl.rcParams['lines.markersize'] ** 2.0)

posterior_big = plot_tsne(trainer, trainer.model, complete_mouse, f"./max_data_model",
                          colors=colors, s=dot_size, edgecolors='black')

# posterior_ebi_annotated = trainer.create_posterior(model, ebi_with_celltypes, indices=np.arange(len(dataset)))
posterior_ebi_annotated = plot_tsne(trainer_big, trainer_big.model, ebi_with_celltypes, f"./plots/small_{tissue}_data_portion_in_big",
#                                    colors=colors, s=dot_size, edgecolors='black')
